In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.utils import class_weight, resample
from keras.utils import to_categorical
from keras.metrics import Precision,Recall
from keras import Model

## EDA & VIZ 

In [ ]:
normal_df = pd.read_csv('/kaggle/input/heartbeat/ptbdb_normal.csv',header=None)
unormal_df = pd.read_csv('/kaggle/input/heartbeat/ptbdb_abnormal.csv',header=None)

In [ ]:
len(normal_df)

In [ ]:
len(unormal_df)

In [ ]:
df=pd.concat([normal_df,unormal_df])
df=df.reset_index(drop=True)
df

In [ ]:
clases = df.iloc[:,187].value_counts()
plt.figure(figsize=(8,8))
plt.pie(clases, labels=['Unormal','Normal'], autopct='%1.1f%%')

In [ ]:
fig, ax = plt.subplots(4, sharex=True, sharey=True)
fig.set_size_inches(10,10)
ax[0].plot(df.iloc[0,:186])
ax[0].set_title("Normal")
ax[1].plot(df.iloc[1,:186])
ax[1].set_title("Normal")
ax[2].plot(df.iloc[-1,:186])
ax[2].set_title("UNormal")
ax[3].plot(df.iloc[-2,:186])
ax[3].set_title("UNormal")

In [ ]:
X = df.iloc[:,:186]
y = df.iloc[:,-1]
trax,tesx,tray,tesy = train_test_split(X,y,test_size=.3,random_state=42,stratify=y)

In [ ]:


x_train = trax.values.reshape(len(trax), trax.shape[1],1)
x_test = tesx.values.reshape(len(tesx), tesx.shape[1],1)

# CNN

In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization,MultiHeadAttention,GlobalAveragePooling1D,LayerNormalization,Input
from keras.models import Sequential

In [ ]:
cnn = Sequential([
    Conv1D(512,3,input_shape=(x_train.shape[1],1), activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(128,3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Conv1D(64,2, activation='relu'),
    BatchNormalization(),
    Conv1D(32,2, activation='selu'),
    GlobalAveragePooling1D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    
])

In [ ]:
cnn.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy',Precision(),Recall()])

In [ ]:
cnn.fit(x_train, tray, epochs=100, batch_size=32, validation_split=.2)

In [ ]:
scores = cnn.evaluate(x_test,tesy)

In [ ]:
pred= cnn.predict(x_test)
yhat = pred>=.5
yhat = yhat.astype(int)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(tesy.values,yhat))

# TRANSFORMER

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = LayerNormalization()(x)
    return x + res

In [ ]:
inputs = Input(shape=(x_train.shape[1],1))
x = inputs
x = transformer_encoder(x, 64, 4, 4, 0.1)
x = GlobalAveragePooling1D()(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="selu")(x)
x = Dense(32, activation="selu")(x)
x = Dense(16, activation="selu")(x)
outputs = Dense(1, activation="sigmoid")(x)
tf_model= Model(inputs, outputs)

In [ ]:
tf_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=["accuracy",Recall(),Precision()]
)

In [ ]:
tf_model.fit(x_train,tray,validation_split=0.2,epochs=100)

In [ ]:
scores = tf_model.evaluate(x_test,tesy)

In [ ]:
pred= tf_model.predict(x_test)
yhat = pred>=.5
yhat = yhat.astype(int)
print(classification_report(tesy.values,yhat))